<a href="https://colab.research.google.com/github/Hohlas/Nero/blob/main/fx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:

#!pip install mpl_finance
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_finance import candlestick_ohlc
# подключение TPU в Colab
import tensorflow as tf
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.TPUStrategy(resolver)

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.models import load_model
from keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Я пишу программу для автоматической торговли на валютном рынке на базе нейронной сети. Скажи пожалуйста, какую нейронную сеть лучше использовать для прогнозирования временных рядов, в частности для финансовых рынков форекс. Опиши подробно, но простым языком как нужно взаимодействовать с библиотекой нейронной сети, какие данные подавать на вход, как их подготавливать, что учитывать при обработке входных данных? Какие данные получаются на выходе нейронной сети, и как их интерпретировать?


---


*лучше всего подходят рекуррентные нейронные сети (RNN). Они хорошо работают с последовательными данными и учитывают зависимости между предыдущими и текущими значениями.
Конкретнее, рекомендую использовать LSTM (долгая краткосрочная память) - это усовершенствованный вид RNN, который может учиться долгосрочным зависимостям(Keras, PyTorch, TensorFlow). На вход подаём подготовленные последовательности цен, выход - прогноз на следующий шаг. Используем алгоритм обратного распространения ошибки*.


---



файл history_data содержит цены часового таймфрейма за последние 10 лет. Но мне не требуется прогноз на каждом баре. Лучше определять точки (моменты входа), в которых цена с высокой вероятностью поменяет свое направление. Чтобы соотношение Profit/Loss в этих точках было максимальным. Достаточно, чтобы точки входа формировались в среднем с интервалом 20 - 30 бар.
Да, приведи пожалуйста пример кода. В качестве входных данных предлагаю использовать массив из последних 100 значений ценового ряда


In [3]:
# Загрузка и normalizация данных
history_data = 'https://drive.google.com/uc?id=1_eYsMYv8L_rrFrNnVN39ugbSVvC12Mm5'
data = pd.read_csv(history_data, header=None, sep=',', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])
data = data[['high', 'low']]
print(f'Rows with missing values: {data.isnull().sum().sum()}') # количество пропусков в каждом столбце
data.dropna(inplace=True) # удаление пустых строк без создания копии данных
print('read ',len(data),'lines in ',history_data)

Rows with missing values: 0
read  155311 lines in  https://drive.google.com/uc?id=1_eYsMYv8L_rrFrNnVN39ugbSVvC12Mm5


In [4]:

# Преобразуем данные в последовательности
X = [] # набор входных последовательностей
y = [] # ответы для обучения

scaler_high = MinMaxScaler(feature_range=(0, 1))
scaler_low = MinMaxScaler(feature_range=(0, 1))
for i in range(100, len(data)):
    window_high = scaler_high.fit_transform(data['high'][i-100:i].values.reshape(-1, 1))
    window_low = scaler_low.fit_transform(data['low'][i-100:i].values.reshape(-1, 1))
    window = np.hstack((window_high, window_low))
    X.append(window)
    y.append(1 if data.iloc[i, 0] > data.iloc[i-1, 0] else 0)
X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 2))

# Разделение данных на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print('X_train, X_val, y_train, y_val:  are ready')
# %% Создание модели LSTM
# Создание обратных вызовов
early_stopping = EarlyStopping(monitor='val_loss', patience=5) # для ранней остановки: останавливает обучение, когда val_loss не улучшается в течение трех эпох (patience=3)
model_checkpoint = ModelCheckpoint('model_{epoch}.h5', save_freq='epoch') # для сохранения модели после каждой эпохи в файл model_{epoch}.h5

with strategy.scope():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 2)))
    model.add(LSTM(units=50))
    model.add(Dense(1, activation='sigmoid'))

    # Компиляция и обучение модели
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, callbacks=[early_stopping, model_checkpoint]) # Обучение модели
    model.save('result_model.h5') # Сохранение последней модели (необязательно, т.к. сохраняются какждую эпоху)
print('Learning complete')

X_train, X_val, y_train, y_val:  are ready
Epoch 1/100
3881/3881 [==============================] - 120s 29ms/step - loss: 0.6887 - accuracy: 0.5416 - val_loss: 0.6859 - val_accuracy: 0.5459
Epoch 2/100
3881/3881 [==============================] - 109s 28ms/step - loss: 0.6720 - accuracy: 0.5597 - val_loss: 0.6699 - val_accuracy: 0.5660
Epoch 3/100
3881/3881 [==============================] - 110s 28ms/step - loss: 0.6673 - accuracy: 0.5700 - val_loss: 0.6672 - val_accuracy: 0.5720
Epoch 4/100
3881/3881 [==============================] - 109s 28ms/step - loss: 0.6655 - accuracy: 0.5730 - val_loss: 0.6659 - val_accuracy: 0.5754
Epoch 5/100
3881/3881 [==============================] - 108s 28ms/step - loss: 0.6637 - accuracy: 0.5774 - val_loss: 0.6639 - val_accuracy: 0.5773
Epoch 6/100
3881/3881 [==============================] - 108s 28ms/step - loss: 0.6620 - accuracy: 0.5805 - val_loss: 0.6685 - val_accuracy: 0.5667
Epoch 7/100
3881/3881 [==============================] - 109s 28ms/st

In [ ]:
# %% Прогнозирование следующего бара
model = load_model('model_6.h5') # загрузка модели с лучшими результатами
last_100_data_high = scaler_high.transform(data['high'][-100:].values.reshape(-1, 1))
last_100_data_low = scaler_low.transform(data['low'][-100:].values.reshape(-1, 1))
last_100_data = np.hstack((last_100_data_high, last_100_data_low))
predicted_price = model.predict(np.reshape(last_100_data, (1, last_100_data.shape[0], 2)))
predicted_direction = 'up' if predicted_price > 0.5 else 'down'
print(f'Предсказанное направление для следующего бара - {predicted_direction}.')

In [ ]:
#     П Е Ч А Т Ь  Г Р А Ф И К А
# Загрузка данных из файла
history_data = 'https://drive.google.com/uc?id=1_eYsMYv8L_rrFrNnVN39ugbSVvC12Mm5'
data = pd.read_csv(history_data, header=None, sep=',', names=['date', 'time', 'open', 'high', 'low', 'close', 'volume'])
data = data.tail(400) # Выбор последних 100 строк
# Выбираем только столбцы 'high' и 'low'
data_hl = data[['high', 'low']]
fig, ax = plt.subplots()
# Строим бары по высоте
ax.bar(range(len(data_hl)), data_hl['high'] - data_hl['low'], bottom=data_hl['low'], width=0.3, color='black')
fig.set_size_inches(20, 5)# Установка размера окна
ax.set_xlabel('Time (index)')
ax.set_ylabel('Price')
ax.set_title('High-Low Bar Chart')